In [26]:
import pm4py
import sys
import random

import pandas as pd
sys.path.append("../../TaskExecutionTimeMining")

from event_log_transformer import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [27]:
file_path = "../../../data/pcr_event_log.csv"
event_log = pd.read_csv(file_path)

event_log['time:timestamp'] = pd.to_datetime(event_log['time:timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

event_log

,concept:instance,id:id,cpee:activity,cpee:instance,lifecycle:transition,cpee:lifecycle:transition,cpee:state,time:timestamp,data,cpee:description,concept:name,concept:endpoint,cpee:activity_uuid,raw,start_timestamp
0,10633,a2,a2,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.616,"{'value': None, 'children': [('', {'value': ''...",NaN,timeout,https://greschner.azurewebsites.net/backend/se...,7231c47e22be012a049f25001506f2d7,NaN,2023-04-04 21:07:16.616000
1,10633,a4,a4,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.627,"{'value': None, 'children': [('', {'value': ''...",NaN,Match patient data,https://greschner.azurewebsites.net/backend/corr,8faebce64254cf3a01782d0c73e3fc14,NaN,2023-04-04 21:07:16.627000
2,10633,a6,a6,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.628,"{'value': None, 'children': [('', {'value': ''...",NaN,Wait for plate validation,https-get://cpee.org/ing/correlators/message/r...,c85b1d6fd598981d3516d70bd5db942d,NaN,2023-04-04 21:07:16.628000
3,10633,a6,a6,791532a2-79c9-4ef0-a09a-0793f010957e,unknown,activity/receiving,NaN,2023-04-04 21:07:16.639,NaN,NaN,Wait for plate validation,https-get://cpee.org/ing/correlators/message/r...,c85b1d6fd598981d3516d70bd5db942d,"{'value': None, 'children': [('', {'value': ''...",2023-04-04 21:07:16.639000
4,10633,a2,a2,791532a2-79c9-4ef0-a09a-0793f010957e,unknown,activity/receiving,NaN,2023-04-04 21:07:16.815,NaN,NaN,timeout,https://greschner.azurewebsites.net/backend/se...,7231c47e22be012a049f25001506f2d7,"{'value': None, 'children': [('', {'value': ''...",2023-04-04 21:07:16.815000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117698,16467,a1,a1,3564fab5-340d-4605-9e10-4512889a24e8,unknown,activity/receiving,NaN,2023-05-17 22:50:13.208,NaN,NaN,Receive sample state,https://mygreschner.com//backend/corr,e6082f8d674b07f40aaa33d50c9d8c89,"{'value': None, 'children': [('', {'value': ''...",2023-05-17 22:50:13.208000
117699,16467,a1,a1,3564fab5-340d-4605-9e10-4512889a24e8,complete,activity/done,NaN,2023-05-17 22:50:13.250,NaN,NaN,Receive sample state,https://mygreschner.com//backend/corr,e6082f8d674b07f40aaa33d50c9d8c89,NaN,2023-05-17 22:50:13.250000
117700,16467,a5,a5,3564fab5-340d-4605-9e10-4512889a24e8,start,activity/calling,NaN,2023-05-17 22:50:13.303,"{'value': None, 'children': [('', {'value': ''...",NaN,Callback timeout,https://mygreschner.com/backend/services/timeout2,f740d70dc17fcb6f48f668891ab9958f,NaN,2023-05-17 22:50:13.303000
117701,16467,a5,a5,3564fab5-340d-4605-9e10-4512889a24e8,unknown,activity/receiving,NaN,2023-05-17 22:50:13.378,NaN,NaN,Callback timeout,https://mygreschner.com/backend/services/timeout2,f740d70dc17fcb6f48f668891ab9958f,NaN,2023-05-17 22:50:13.378000


In [28]:
event_log_pruned = event_log.copy()
event_log_pruned = event_log_pruned[event_log_pruned['lifecycle:transition'] != 'unknown']
event_log_pruned

,concept:instance,id:id,cpee:activity,cpee:instance,lifecycle:transition,cpee:lifecycle:transition,cpee:state,time:timestamp,data,cpee:description,concept:name,concept:endpoint,cpee:activity_uuid,raw,start_timestamp
0,10633,a2,a2,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.616,"{'value': None, 'children': [('', {'value': ''...",NaN,timeout,https://greschner.azurewebsites.net/backend/se...,7231c47e22be012a049f25001506f2d7,NaN,2023-04-04 21:07:16.616000
1,10633,a4,a4,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.627,"{'value': None, 'children': [('', {'value': ''...",NaN,Match patient data,https://greschner.azurewebsites.net/backend/corr,8faebce64254cf3a01782d0c73e3fc14,NaN,2023-04-04 21:07:16.627000
2,10633,a6,a6,791532a2-79c9-4ef0-a09a-0793f010957e,start,activity/calling,NaN,2023-04-04 21:07:16.628,"{'value': None, 'children': [('', {'value': ''...",NaN,Wait for plate validation,https-get://cpee.org/ing/correlators/message/r...,c85b1d6fd598981d3516d70bd5db942d,NaN,2023-04-04 21:07:16.628000
6,10633,a4,a4,791532a2-79c9-4ef0-a09a-0793f010957e,complete,activity/done,NaN,2023-04-04 21:07:25.115,NaN,NaN,Match patient data,https://greschner.azurewebsites.net/backend/corr,8faebce64254cf3a01782d0c73e3fc14,NaN,2023-04-04 21:07:25.115000
8,10633,a6,a6,791532a2-79c9-4ef0-a09a-0793f010957e,complete,activity/done,NaN,2023-04-05 05:03:06.716,NaN,NaN,Wait for plate validation,https-get://cpee.org/ing/correlators/message/r...,c85b1d6fd598981d3516d70bd5db942d,NaN,2023-04-05 05:03:06.716000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117694,16467,a1,a1,3564fab5-340d-4605-9e10-4512889a24e8,start,activity/calling,NaN,2023-05-17 22:49:52.543,"{'value': None, 'children': [('', {'value': ''...",NaN,Receive sample state,https://mygreschner.com//backend/corr,e6082f8d674b07f40aaa33d50c9d8c89,NaN,2023-05-17 22:49:52.543000
117695,16467,a6,a6,3564fab5-340d-4605-9e10-4512889a24e8,complete,activity/done,NaN,2023-05-17 22:49:52.513,NaN,NaN,Wait for plate validation,https-get://cpee.org/ing/correlators/message/r...,e076f64d6fce97fad7f83b70f7c7263e,NaN,2023-05-17 22:49:52.513000
117699,16467,a1,a1,3564fab5-340d-4605-9e10-4512889a24e8,complete,activity/done,NaN,2023-05-17 22:50:13.250,NaN,NaN,Receive sample state,https://mygreschner.com//backend/corr,e6082f8d674b07f40aaa33d50c9d8c89,NaN,2023-05-17 22:50:13.250000
117700,16467,a5,a5,3564fab5-340d-4605-9e10-4512889a24e8,start,activity/calling,NaN,2023-05-17 22:50:13.303,"{'value': None, 'children': [('', {'value': ''...",NaN,Callback timeout,https://mygreschner.com/backend/services/timeout2,f740d70dc17fcb6f48f668891ab9958f,NaN,2023-05-17 22:50:13.303000


In [29]:
start_end_event_log2 = TransformEventLog.start_end_event_log_mult(event_log_pruned,
                                                                 start_name_1 = 'start',
                                                                 complete_name_1 = 'complete',
                                                                 merge_activity_on = ['concept:instance', 'concept:name'],
                                                                 unique_column='cpee:activity_uuid')
start_end_event_log2 = TransformEventLog.seconds_in_day(start_end_event_log2, 'time:timestamp_start')
start_end_event_log2 = TransformEventLog.day_of_week(start_end_event_log2, 'time:timestamp_start')

start_end_event_log2

/Users/ferdinandfuchs/Documents/TaskExecutionTimeMining-gamlss/src/notebooks/PCR/../../TaskExecutionTimeMining/event_log_transformer.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + complete_name_gen] - start_end_event_log[timestamp_name + start_name_gen]
/Users/ferdinandfuchs/Documents/TaskExecutionTimeMining-gamlss/src/notebooks/PCR/../../TaskExecutionTimeMining/event_log_transformer.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,concept:instance,id:id_start,cpee:activity_start,cpee:instance_start,lifecycle:transition_start,cpee:lifecycle:transition_start,cpee:state_start,time:timestamp_start,data_start,cpee:description_start,concept:name,concept:endpoint_start,cpee:activity_uuid_start,raw_start,start_timestamp_start,id:id_complete,cpee:activity_complete,cpee:instance_complete,lifecycle:transition_complete,cpee:lifecycle:transition_complete,cpee:state_complete,time:timestamp_complete,data_complete,cpee:description_complete,concept:endpoint_complete,cpee:activity_uuid_complete,raw_complete,start_timestamp_complete,duration,duration_seconds,duration_ms,duration_hours,seconds_in_day,day_of_week
11369,16495,a3,a3,6bf7c694-35d2-4134-a35d-f52eaa51d658,start,activity/calling,NaN,2023-05-20 14:43:28.526,"{'value': None, 'children': [('', {'value': ''...",NaN,Send notification,https://mygreschner.com/backend/services/notif...,00000f925eff6cf9e2dd9f7b869cc332,NaN,2023-05-20 14:43:28.526000,a3,a3,6bf7c694-35d2-4134-a35d-f52eaa51d658,complete,activity/done,NaN,2023-05-20 14:43:28.819,NaN,NaN,https://mygreschner.com/backend/services/notif...,00000f925eff6cf9e2dd9f7b869cc332,NaN,2023-05-20 14:43:28.819000,0 days 00:00:00.293000,0.293,293.0,0.000081,53008,5
101623,14812,a4,a4,786569b6-6fd4-4fe2-a5f4-20c4f39f3307,start,activity/calling,NaN,2023-04-28 13:55:37.424,"{'value': None, 'children': [('', {'value': ''...",NaN,Match patient data,https://mygreschner.com//backend/corr,0003aecdd0133b1287373b4e5aae113e,NaN,2023-04-28 13:55:37.424000,a4,a4,786569b6-6fd4-4fe2-a5f4-20c4f39f3307,complete,activity/done,NaN,2023-04-28 13:55:41.064,NaN,NaN,https://mygreschner.com//backend/corr,0003aecdd0133b1287373b4e5aae113e,NaN,2023-04-28 13:55:41.064000,0 days 00:00:03.640000,3.640,3640.0,0.001011,50137,4
79114,13597,a2,a2,72de7428-a85d-4ee2-9c71-556eaa2bb0b5,start,activity/calling,NaN,2023-04-20 17:12:02.554,"{'value': None, 'children': [('', {'value': ''...",NaN,timeout,https://mygreschner.com/backend/services/timeout2,00041cf593ad0f089e27119c34a33695,NaN,2023-04-20 17:12:02.554000,a2,a2,72de7428-a85d-4ee2-9c71-556eaa2bb0b5,complete,activity/done,NaN,2023-04-20 19:44:32.228,NaN,NaN,https://mygreschner.com/backend/services/timeout2,00041cf593ad0f089e27119c34a33695,NaN,2023-04-20 19:44:32.228000,0 days 02:32:29.674000,9149.674,9149674.0,2.541576,61922,3
98538,14616,a4,a4,6d1b2d51-5ae1-46bd-bdae-eb5e04227a28,start,activity/calling,NaN,2023-04-27 13:53:05.267,"{'value': None, 'children': [('', {'value': ''...",NaN,Match patient data,https://mygreschner.com//backend/corr,0007e9d7b011e68f2030ea31c19feb76,NaN,2023-04-27 13:53:05.267000,a4,a4,6d1b2d51-5ae1-46bd-bdae-eb5e04227a28,complete,activity/done,NaN,2023-04-27 13:53:09.125,NaN,NaN,https://mygreschner.com//backend/corr,0007e9d7b011e68f2030ea31c19feb76,NaN,2023-04-27 13:53:09.125000,0 days 00:00:03.858000,3.858,3858.0,0.001072,49985,3
34337,10320,a2,a2,5778bdc7-3d1f-443c-bb24-8dd8e6728b3d,start,activity/calling,NaN,2023-04-03 17:58:20.376,"{'value': None, 'children': [('', {'value': ''...",NaN,timeout,https://greschner.azurewebsites.net/backend/se...,0008a001328985e9d9133e7a55876092,NaN,2023-04-03 17:58:20.376000,a2,a2,5778bdc7-3d1f-443c-bb24-8dd8e6728b3d,complete,activity/done,NaN,2023-04-03 20:36:42.986,NaN,NaN,https://greschner.azurewebsites.net/backend/se...,0008a001328985e9d9133e7a55876092,NaN,2023-04-03 20:36:42.986000,0 days 02:38:22.610000,9502.610,9502610.0,2.639614,64700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40720,11283,a4,a4,1c84f4b5-74a3-44b9-b268-01f0ef120814,start,activity/calling,NaN,2023-04-09 14:21:09.416,"{'value': None, 'children': [('', {'value': ''...",NaN,Match patient data,https://greschner.azurewebsites.net/backend/corr,fff0c6eb141ad52b940ffe1639128722,NaN,2023-04-09 14:21:09.416000,a4,a4,1c84f4b5-74a3-44b9-b268-01f0ef120814,complete,activity/done,NaN,2023-04-09 14:21:48.264,NaN,NaN,https://greschner.azurewebs

In [30]:
start_end_event_log2 = TransformEventLog.value_count_per_case_without_lifecycle(start_end_event_log2,
                                                             column_name = 'concept:name',
                                                            case_name = 'concept:instance',
                                                            timestamp_name = 'time:timestamp_start',
                                                            unique_id = 'cpee:activity_uuid_start')
start_end_event_log2 = start_end_event_log2.sort_values(by='time:timestamp_start')


/Users/ferdinandfuchs/Documents/TaskExecutionTimeMining-gamlss/src/notebooks/PCR/../../TaskExecutionTimeMining/event_log_transformer.py:171: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  value_count_event_log = value_count_event_log.fillna(0)


In [31]:
start_end_event_log2 = start_end_event_log2.rename(columns={'concept:instance': 'case:concept:name'})


In [32]:
unique_values = start_end_event_log2["concept:name"].unique()
print(unique_values)


['timeout' 'Wait for plate validation' 'Match patient data'
 'Receive sample state' 'Callback timeout' 'Export to EMS' 'Export result'
 'Send notification']


In [33]:
random.seed(42)

train, test = pm4py.ml.split_train_test(start_end_event_log2, case_id_key='case:concept:name')

In [34]:
print(train.shape, test.shape)
print(train['case:concept:name'].unique().shape, test['case:concept:name'].unique().shape)

(26051, 42) (6309, 42)
(4960,) (1202,)


In [35]:
train.to_csv('../../../data/PCR/transformed_event_logs/PCR_train.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/PCR/transformed_event_logs/PCR_train.pickle')

In [36]:
test.to_csv('../../../data/PCR/transformed_event_logs/PCR_test.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')
train.to_pickle('../../../data/PCR/transformed_event_logs/PCR_test.pickle')